In [1]:
import pandas as pd
from SyntheticControlMethods import Synth
import random
import numpy as np
from sklearn.neighbors import KDTree
import cvxpy as cvx
from scipy.optimize import minimize, differential_evolution
import pickle

In [98]:
treated_unit= '81959733'
treated_period = 2013
id_var = 'pidp'
time_var = 'year'
outcome_var = 'ind_inc_deflated'

In [99]:
df = pd.read_csv('./test_data.csv', index_col=0)
df['pidp'] = df['pidp'].astype(str)
df.dtypes

year                float64
pidp                 object
asian               float64
black               float64
dvage               float64
ind_inc_deflated    float64
low                 float64
mastat_recoded      float64
middle              float64
mixed               float64
other               float64
dtype: object

In [100]:
t_u = df[df.pidp==treated_unit].copy()

In [101]:
c_u = df[df.pidp!=treated_unit].copy()

In [102]:
selection = random.sample(c_u.pidp.unique().tolist(), 100)

In [103]:
sample = df[df.pidp.isin(selection)]
data = pd.concat([sample,t_u], axis=0)

In [104]:
sc = Synth(dataset=data, outcome_var=outcome_var, id_var=id_var, time_var=time_var, treatment_period=treated_period, treated_unit=treated_unit)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.39131D+06    |proj g|=  8.88889D-01

At iterate    3    f=  1.50136D+06    |proj g|=  9.25140D-01

At iterate    6    f=  1.50107D+06    |proj g|=  4.42378D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9      6      8     12     0     0   4.424D-01   1.501D+06
  F =   1501069.1430513542     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    6    f=  1.28033D+05    |proj g|=  8.82161D-01



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    9    f=  1.20181D+05    |proj g|=  1.00000D+00


/home/daniel/code/costofcare/lib64/python3.11/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     12    175     38     1     5   7.825D-01   1.186D+05
  F =   118637.52482522048     

ABNORMAL_TERMINATION_IN_LNSRCH                              
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.47129D+06    |proj g|=  9.99925D-01

At iterate    3    f=  1.76680D+06    |proj g|=  4.49762D-01

At iterate    6    f=  1.50108D+06    |proj g|=  9.99996D-01

At iterate    9    f=  1.50107D+06    |proj g|=  0.00000D+00

       

/home/daniel/code/costofcare/lib64/python3.11/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(



At iterate    3    f=  1.17528D+05    |proj g|=  9.29442D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9      3     13     10     0     5   9.294D-01   1.175D+05
  F =   117527.82333259824     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.44709D+06    |proj g|=  9.76605D-01

At iterate    3    f=  1.50107D+06    |proj g|=  2.49375D-01

           * * *

Tit   = total number of iterations
Tnf   = total nu

In [11]:
def get_control_matrices_b(data_object, k_n):
    data = data_object['data'].copy()
    ncol = data.shape[1] - 1
    sample_weights = data_object['weight'].copy()
    data.index.names = ['var', 'year']
    t_time = data_object['treat_time']
    target_var = data_object['target_var']
    data.index = data.index.map(lambda idx: (idx[0], idx[1] - t_time))
    sample_weights.index = sample_weights.index - t_time
    data = data.sort_index(ascending=True).copy()
    df_T0 = data.loc[pd.IndexSlice[:, :-1], :]
    if ncol < k_n:
        k_n = ncol
    try:
        kdt = KDTree(df_T0.T, leaf_size=30, metric='euclidean')
    except ValueError:
        return None
    idx = kdt.query(df_T0.T, k=k_n, return_distance=False)[0, :]
    select_data = data.iloc[:, idx]
    select_df_T0 = df_T0.iloc[:, idx]
    c_out_all = np.array([x for x in select_data.iloc[:,1:].loc[[target_var]].values])
    c_out = np.array([x for x in select_df_T0.iloc[:,1:].loc[[target_var]].values])
    l_means = select_df_T0.iloc[:,1:].sort_index().groupby(level='var').mean()
    c_cov_mean = np.array([x for x in l_means.values])
    return c_out_all, c_out, c_cov_mean

In [12]:
treated_unit= '81959733'
treated_period = 2014
id_var = 'pidp'
time_var = 'year'
outcome_var = 'ind_inc_deflated'
n_covariates = 9

In [13]:
with open('data.pkl', 'rb') as file:
    data_object = pickle.load(file)

In [28]:
t_u = data[data.pidp == '81959733'].pivot(index='pidp', columns='year').T
c_u = data[data.pidp != '81959733'].pivot(index='pidp', columns='year').T
sample = pd.concat([t_u, c_u], axis=1, join="inner") 

In [29]:
data_object['data'] = sample

In [88]:
class Optimizer(object):
    def __init__(self, data_object):
        self.data_object = data_object
        self.data = self.data_object['data'].copy()
        self.sample_weights = self.data_object['weight'].copy()
        self.data.index.names = ['var', 'year']
        self.t_time = self.data_object['treat_time']
        self.target_var = self.data_object['target_var']
        self.n_covariates = len(self.data.index.get_level_values(0).unique())
        self.min_loss = float("inf")
        self.random_seed = 0
        self.rng = np.random.default_rng(self.random_seed)
        self.steps = 8
        self.w = None
        self.v = None
        self.random_seed = 0
        self.control_outcome_all = None
        self.control_outcome = None
        self.unscaled_control_covariates = None
        self.synth_outcome = None
        self.synth_covariates = None
        self.unscaled_treated_covariates = None
        self.treated_outcome_all = None
        self.treated_outcome = None
        self.treated_covariates = None
        self.control_covariates = None
        self.get_control_matrices()
        self.get_treat_matrices()
        self.rescale_covariate_variance()
    
    def get_control_matrices(self):
        self.control_outcome_all = np.array([x for x in self.data.iloc[:,1:].loc[[self.target_var]].values])
        self.control_outcome = np.array([x for x in self.data.iloc[:,1:].loc[[self.target_var]].loc[(slice(None), slice(None, self.t_time-1)), :].values])
        l_means = self.data.iloc[:,1:].sort_index().loc[(slice(None), slice(None, self.t_time-1)), :].groupby(level='var').mean()
        self.unscaled_control_covariates = np.array([x for x in l_means.values])

    def get_treat_matrices(self):
        self.treated_outcome_all = np.array([[x] for x in self.data.iloc[:,0][self.target_var].values])
        self.treated_outcome = np.array([[x] for x in self.data.iloc[:,0][self.target_var][:self.t_time-1].values])
        l_means = self.data.iloc[:,:1].sort_index().loc[(slice(None), slice(None, self.t_time-1)), :].groupby(level='var').mean().mean(axis=1)
        self.unscaled_treated_covariates = np.array([[x] for x in l_means.values])

    def rescale_covariate_variance(self):
        big_dataframe = np.concatenate((self.unscaled_treated_covariates, self.unscaled_control_covariates), axis=1)
        big_dataframe /= np.apply_along_axis(np.std, 0, big_dataframe)
        self.treated_covariates = big_dataframe[:,0].reshape(self.n_covariates, 1)
        self.control_covariates = big_dataframe[:,1:]
    
    def optimize(self):

        args = (self.treated_outcome,
                self.treated_covariates,
                self.control_outcome,
                self.control_covariates
                )
        
        for step in range(self.steps):
            if step == 0:
                v_0 = np.full(self.n_covariates, 1/self.n_covariates)
            else:
                v_0 = self.rng.dirichlet(np.ones(self.n_covariates), size=1)

            bnds = tuple((0,1) for _ in range(self.n_covariates))
            self.res = minimize(self.total_loss, v_0,  args=(args),
                                method='L-BFGS-B', bounds=bnds,
                                options={'gtol': 1e-8,'disp':3, 'iprint':3})
            self.res_x = self.res.x
    
    def total_loss(self,
                   v_0,
                   treated_outcome,
                   treated_covariates,
                   control_outcome,
                   control_covariates):
        
        n_controls = control_outcome.shape[1]
        V = np.diag(v_0)
        w = cvx.Variable((n_controls, 1), nonneg=True)
        treated_synth_difference = cvx.sum(V @ cvx.square(treated_covariates.T - control_covariates @ w))
        objective = cvx.Minimize(treated_synth_difference)
        constraints = [cvx.sum(w) == 1]
        problem = cvx.Problem(objective, constraints)
        
        try:
            result = problem.solve(verbose=False)
            loss = (treated_outcome - control_outcome @ w.value).T @ (treated_outcome - control_outcome @ w.value)
        except:
            return float(np.inf)

        if loss < self.min_loss:
            self.min_loss = loss
            self.w = w.value
            self.v = np.diagonal(V) / np.sum(np.diagonal(V))
            self.synth_outcome = self.w.T @ self.control_outcome_all.T
            self.synth_covariates = self.control_covariates @ self.w
        return loss
        

In [89]:
result = Optimizer(data_object)

In [90]:
result.optimize()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.48899D+07    |proj g|=  8.88889D-01

At iterate    3    f=  1.16961D+06    |proj g|=  1.21318D-01

At iterate    6    f=  1.12277D+06    |proj g|=  9.16120D-02


/tmp/ipykernel_20329/1123782311.py:64: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  self.res = minimize(self.total_loss, v_0,  args=(args),



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9      7      9     12     0     3   3.260D-01   1.123D+06
  F =   1122767.8429995503     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.61512D+06    |proj g|=  9.97569D-01

At iterate    3    f=  1.40677D+06    |proj g|=  9.98923D-01

At iterate    6    f=  1.40631D+06    |proj g|=  9.98922D-01
  ys=-4.290E+01  -gs= 4.073E+03 BFGS update SKIPPED
  ys=-3.550E-01  -


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



At iterate   15    f=  1.11947D+06    |proj g|=  1.16415D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     15    128     23     3     0   1.164D-01   1.119D+06
  F =   1119470.9020986350     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.70224D+06    |proj g|=  9.99925D-01

At iterate    3    f=  1.12277D+06    |proj g|=  9.86864D-01

           * * *

Tit   = total number of iterations
Tnf   = total nu

In [105]:
result.min_loss

array([[1119470.90209863]])

In [106]:
sc.original_data.min_loss

array([[117527.82333086]])

In [96]:
data

,year,pidp,asian,black,dvage,ind_inc_deflated,low,mastat_recoded,middle,mixed,other
603,2010.0,68571207,0.0,0.0,70.0,0.000000,1.0,1.0,0.0,0.0,0.0
604,2011.0,68571207,0.0,0.0,71.0,0.000000,1.0,1.0,0.0,0.0,0.0
605,2012.0,68571207,0.0,0.0,72.0,0.000000,1.0,1.0,0.0,0.0,0.0
606,2013.0,68571207,0.0,0.0,73.0,0.000000,1.0,1.0,0.0,0.0,0.0
607,2014.0,68571207,0.0,0.0,75.0,0.000000,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1480,2014.0,81959733,0.0,0.0,23.0,968.718466,0.0,1.0,0.0,0.0,0.0
1481,2015.0,81959733,0.0,0.0,24.0,0.000000,0.0,1.0,0.0,0.0,0.0
1482,2016.0,81959733,0.0,0.0,25.0,0.000000,0.0,1.0,0.0,0.0,0.0
1483,2017.0,81959733,0.0,0.0,26.0,0.000000,0.0,1.0,0.0,0.0,0.0


In [97]:
data_object['data']

pidp          81959733  1156396447  1157758495  1224193807  1224399847  \
      year                                                               
asian 2010.0       0.0         0.0         1.0         0.0         0.0   
      2011.0       0.0         0.0         1.0         0.0         0.0   
      2012.0       0.0         0.0         1.0         0.0         0.0   
      2013.0       0.0         0.0         1.0         0.0         0.0   
      2014.0       0.0         0.0         1.0         0.0         0.0   
...                ...         ...         ...         ...         ...   
other 2014.0       0.0         0.0         0.0         0.0         0.0   
      2015.0       0.0         0.0         0.0         0.0         0.0   
      2016.0       0.0         0.0         0.0         0.0         0.0   
      2017.0       0.0         0.0         0.0         0.0         0.0   
      2018.0       0.0         0.0         0.0         0.0         0.0   

pidp          1292191091  1360093847  1360438607  1360828247  136330499  ...  \
      year                                                               ...   
asian 2010.0         1.0         0.0         0.0         0.0        0.0  ...   
      2011.0         1.0         0.0         0.0         0.0        0.0  ...   
      2012.0         1.0         0.0         0.0         0.0        0.0  ...   
      2013.0         1.0         0.0         0.0         0.0        0.0  ...   
      2014.0         1.0         0.0         0.0         0.0        0.0  ...   
...                  ...         ...         ...         ...        ...  ...   
other 2014.0         0.0         0.0         0.0         0.0        0.0  ...   
      2015.0         0.0         0.0         0.0         0.0        0.0  ...   
      2016.0         0.0         0.0         0.0         0.0        0.0  ...   
      2017.0         0.0         0.0         0.0         0.0        0.0  ...   
      2018.0         0.0         0.0         0.0         0.0        0.0  ...   

pidp          884697011  885176407  885415767  885667371  885797927  89037165  \
      year                                                                      
asian 2010.0        0.0        0.0        1.0        0.0        0.0       0.0   
      2011.0        0.0        0.0        1.0        0.0        0.0       0.0   
      2012.0        0.0        0.0        1.0        0.0        0.0       0.0   
      2013.0        0.0        0.0        1.0        0.0        0.0       0.0   
      2014.0        0.0        0.0        1.0        0.0        0.0       0.0   
...                 ...        ...        ...        ...        ...       ...   
other 2014.0        0.0        0.0        0.0        0.0        0.0       0.0   
      2015.0        0.0        0.0        0.0        0.0        0.0       0.0   
      2016.0        0.0        0.0        0.0        0.0        0.0       0.0   
      2017.0        0.0        0.0        0.0        0.0        0.0       0.0   
      2018.0        0.0        0.0        0.0        0.0        0.0       0.0   

pidp          952716047  953915567  953924411  96145209  
      year                                               
asian 2010.0        0.0        1.0        1.0       0.0  
      2011.0        0.0        1.0        1.0       0.0  
      2012.0        0.0        1.0        1.0       0.0  
      2013.0        0.0        1.0        1.0       0.0  
      2014.0        0.0        1.0        1.0       0.0  
...                 ...        ...        ...       ...  
other 2014.0        0.0        0.0        0.0       0.0  
      2015.0        0.0        0.0        0.0       0.0  
      2016.0        0.0        0.0        0.0       0.0  
      2017.0        0.0        0.0        0.0       0.0  
      2018.0        0.0        0.0        0.0       0.0  

[81 rows x 101 columns]